<h1><center>Metody uczenia maszynowego</center></h1>
<h3><center><ul style="list-style: none;">
    <li>Patrycja Szczakowska ???</li>
    <li>Maciej Majchrowski ???</li>
    <li>Mateusz Gałasiński 234054</li>
    <li>Jakub Kurek 234078</li>
    </ul></center></h3> 
<h2><center>Zadanie 2 - Klastryzacja danych - uczenie nienadzorowane</center></h2>

### Zależności
*Dodatkowe zewnętrzne biblioteki, instalowane dzięki IPython "magic commands"*

In [ ]:
# install additional packages
!pip install k3d > /dev/null

In [ ]:
from pathlib import Path
import matplotlib
import matplotlib.pyplot as plt
from pandas import read_csv
import json
%matplotlib notebook

## Konfiguracja

In [ ]:
class Config(object):
    def __init__(self, dataSourceUrl, dataDelimiter, quantitativeColumns, qualitativeColumns):
        self.dataSourceUrl = dataSourceUrl
        self.dataDelimiter = dataDelimiter
        self.quantitativeColumns = quantitativeColumns
        self.qualitativeColumns = qualitativeColumns
def as_config(dct):
    return Config(
        dct['dataSourceUrl'],
        dct['dataDelimiter'],
        dct['quantitativeColumns'],
        dct['qualitativeColumns'],)
json_config ="""
{
    "dataSourceUrl": "/home/jovyan/host-note/data/winequality-white.csv",
    "dataDelimiter": ";",
    "quantitativeColumns": [
        "season"
    ],
    "qualitativeColumns": [
        "age"
    ]
}"""
config = json.loads(json_config, object_hook = as_config)

## Opis zbiorów danych

In [ ]:
for line in [line for line in Path(config.dataSourceUrl).read_text().splitlines()[:20] if line]:
    print(line)

## Wczytanie zbioru danych

In [ ]:
# from pandas.plotting import scatter_matrix

def load_dataset():
    global dataset
    dataset = read_csv(config.dataSourceUrl, delimiter=config.dataDelimiter, header=0)
#     global qualitativeDataset = dataset[config.qualitativeColumns]
#     global quantitativeDataset = dataset[config.quantitativeColumns]
#     quantitativeDataset.corr()
#     qualitativeDataset.mode()
    # dataset.plot(kind='box', subplots=True, sharex=False, sharey=False)
    # pyplot.show()

    # scatter_matrix(dataset)
    # pyplot.show()
    print(dataset.shape)
    dataset.describe()

# Pętla aplikacji

In [ ]:
import sklearn
from uuid import uuid4
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score
from sklearn.metrics import calinski_harabasz_score, pairwise_distances, davies_bouldin_score, adjusted_rand_score, adjusted_mutual_info_score, homogeneity_score, completeness_score, homogeneity_completeness_v_measure, v_measure_score, silhouette_score
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering, OPTICS, cluster_optics_dbscan
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler

load_dataset()

array = dataset.values
x = array[:,0:len(dataset.columns)-1]
x = StandardScaler().fit_transform(x)
labels_true = array[:,len(dataset.columns)-1]

### Define algorithms
models = []
models.extend([
    (f"AgglomerativeClustering-{linkage}", AgglomerativeClustering(linkage=linkage, n_clusters=10), 0)
    for linkage in ['ward', 'average', 'complete', 'single']])
models.extend([
    (f"GaussianMixture-{covariance_type}", GaussianMixture(covariance_type=covariance_type), 0)
    for covariance_type in ['full', 'tied', 'diag', 'spherical']])
models.extend([
    ('DBSCAN', DBSCAN(eps=0.1, min_samples=2), 0),
    ('OPTICS', OPTICS(min_samples=50, xi=.05, min_cluster_size=.05), 0),
    ('KNN', KMeans(n_clusters=3, random_state=1), 0),
])

### Run for all algorithms
results = []
names = []
for name, model, subplot_row in models:
    print(f"---------------------------\nRunning clustering for: {name}")
    labels = []
    if 'GaussianMixture' in name:
        labels = model.fit_predict(x)
    elif 'OPTICS' in name:
        clustering = model.fit(x)
        labels = cluster_optics_dbscan(reachability=clustering.reachability_,
                                   core_distances=clustering.core_distances_,
                                   ordering=clustering.ordering_, eps=0.5)
    else:
        clustering = model.fit(x)
        labels=clustering.labels_
### Results 
    print("Homogeneity: %0.3f" % homogeneity_score(labels_true, labels))
    print("Completeness: %0.3f" % completeness_score(labels_true, labels))
    print("V-measure: %0.3f" % v_measure_score(labels_true, labels))
    print("Adjusted Rand Index: %0.3f" % adjusted_rand_score(labels_true, labels))
    print("Adjusted Mutual Information: %0.3f" % adjusted_mutual_info_score(labels_true, labels))
    if len(set(labels)) > 1:
        print(f"Quality: {davies_bouldin_score(x, labels)}")
        print("Silhouette Coefficient: %0.3f" % silhouette_score(x, labels))
    print(f"n_clusters_: {len(set(labels)) - (1 if -1 in labels else 0)}")
    print(f"n_noise_: {list(labels).count(-1)}")

### Plots
    # Black removed and is used for noise instead.
    unique_labels = set(labels)
    colors = [plt.cm.Spectral(each)
              for each in np.linspace(0, 1, len(unique_labels))]
    core_samples_mask = np.zeros_like(labels, dtype=bool)
    if name == "DBSCAN":
        core_samples_mask[clustering.core_sample_indices_] = True
    for k, col in zip(unique_labels, colors):
        if k == -1:
            # Black used for noise.
            col = [0, 0, 0, 1]

        class_member_mask = (labels == k)

        xy = x[class_member_mask & core_samples_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                 markeredgecolor='k', markersize=14)

#         xy = x[class_member_mask & ~core_samples_mask]
#         plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
#                  markeredgecolor='k', markersize=6)
    plt.title(name)
    plt.show()